In [1]:
import numpy as np
from hyperopt import hp, fmin, tpe, space_eval
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv('equations_static.csv')
data

,Reaction Name,Reactant(s),Product(s),Reaction Rate
0,NaN,NaN,NaN,NaN
1,PA1_Chemical_Dilution:,PA1_Chemical,NaN,PA1_Chemical * PA1_Chemical_dilution_rate
2,PA2_Chemical_Dilution:,PA2_Chemical,NaN,PA2_Chemical * PA2_Chemical_dilution_rate
3,PA3_Chemical_Dilution:,PA3_Chemical,NaN,PA3_Chemical * PA3_Chemical_dilution_rate
4,PAF_Chemical_Dilution:,PAF_Chemical,NaN,PAF_Chemical * PAF_Chemical_dilution_rate
...,...,...,...,...
57,PapB_Protein_Folded_Catalyzed_Biosynthesis:,PA1_Chemical + PapB_Protein_Folded,PA2_Chemical + PapB_Protein_Folded,PapB_Protein_Folded_kcat * ((PA1_Chemical / Av...
58,PapC_Protein_Folded_Catalyzed_Biosynthesis:,PapC_Protein_Folded + PA2_Chemical,PapC_Protein_Folded + PA3_Chemical,PapC_Protein_Folded_kcat * PapC_Protein_Folded...
59,Deaminase_Catalyzed_Biosynthesis:,Deaminase + PA3_Chemical,Deaminase + PAF_Chemical,Deaminase_kcat * Deaminase * ((PA3_Chemical / ...
60,LAAO_Protein_Folded_Catalyzed_Biosynthesis:,LAAO_Protein_Folded + PAF_Chemical,LAAO_Protein_Folded + PACA_Chemical_Internal,LAAO_Protein_Folded_kcat * LAAO_Protein_Folded...


In [13]:
data[35:45]

,Reaction Name,Reactant(s),Product(s),Reaction Rate
35,Module_Pap_Operon_PapC_Fol_Degradation:,PapC_Protein_Folded,NaN,PapC_Protein_Folded * Module_Pap_Operon_PapC_p...
36,Module_LAAO_LAAO_Txn:,NaN,LAAO_mRNA,Promoter2 * Module_LAAO_LAAO_mRNA_production_r...
37,Module_LAAO_LAAO_Dilution:,LAAO_mRNA,NaN,LAAO_mRNA * Module_LAAO_LAAO_mRNA_dilution_rate
38,Module_LAAO_LAAO_Degradation:,LAAO_mRNA,NaN,LAAO_mRNA * Module_LAAO_LAAO_mRNA_degradation_...
39,Module_LAAO_LAAO_Tsn:,LAAO_mRNA,LAAO_mRNA + LAAO_Protein_Unfolded,LAAO_mRNA * Module_LAAO_LAAO_protein_productio...
40,Module_LAAO_LAAO_Folding:,LAAO_Protein_Unfolded,LAAO_Protein_Folded,LAAO_Protein_Unfolded * Module_LAAO_LAAO_prote...
41,Module_LAAO_LAAO_Unf_Dilution:,LAAO_Protein_Unfolded,NaN,LAAO_Protein_Unfolded * Module_LAAO_LAAO_prote...
42,Module_LAAO_LAAO_Fol_Dilution:,LAAO_Protein_Folded,NaN,LAAO_Protein_Folded * Module_LAAO_LAAO_protein...
43,Module_LAAO_LAAO_Unf_Degradation:,LAAO_Protein_Unfolded,NaN,LAAO_Protein_Unfolded * Module_LAAO_LAAO_prote...
44,Module_LAAO_LAAO_Fol_Degradation:,LAAO_Protein_Folded,NaN,LAAO_Protein_Folded * Module_LAAO_LAAO_protein...
